In [2]:
import numpy as np
import pandas as pd

# ---------------------------------------------------
# Drag coefficient parameterization (Large & Pond)
# ---------------------------------------------------
def Cd_large_pond(U):
    """
    Wind-speed-dependent drag coefficient (neutral stability).
    U in m/s, returns Cd (dimensionless).
    """
    U = np.asarray(U)
    Cd = np.zeros_like(U, dtype=float)

    Cd[U < 11.0] = 1.2e-3
    mid = (U >= 11.0) & (U <= 25.0)
    Cd[mid] = (0.49 + 0.065 * U[mid]) * 1e-3
    Cd[U > 25.0] = 2.6e-3

    return Cd


# ---------------------------------------------------
# Iterative solver for required wind speed
# ---------------------------------------------------
def wind_speed_for_setup_variable_Cd(
    df,
    eta_target_m=2.0,
    rho_w=1025.0,
    rho_air=1.225,
    g=9.81,
    U_init=15.0,
    tol=1e-4,
    max_iter=100
):
    """
    Solve for wind speed required to produce eta_target_m of setup
    using speed-dependent Cd(U).

    Requires columns: L_m, h_m
    """
    out = df.copy()

    L = pd.to_numeric(out["L_m"], errors="coerce").to_numpy()
    h = pd.to_numeric(out["h_m"], errors="coerce").to_numpy()

    # Initial guess
    U = np.full_like(L, U_init, dtype=float)

    for _ in range(max_iter):
        Cd = Cd_large_pond(U)
        U_new = np.sqrt((eta_target_m * rho_w * g * h) /
                        (rho_air * Cd * L))

        if np.nanmax(np.abs(U_new - U)) < tol:
            break

        U = U_new

    out["U_required_mps"] = U
    out["Cd_at_solution"] = Cd_large_pond(U)
    out["U_required_kt"] = out["U_required_mps"] / 0.514444
    out["eta_target_m"] = eta_target_m

    return out


# ---------------------------------------------------
# Main script
# ---------------------------------------------------
in_csv  = "lagoon_list.csv"
out_csv = "lagoon_wind_for_2m_setup_variableCd.csv"

df = pd.read_csv(in_csv)

df_out = wind_speed_for_setup_variable_Cd(
    df,
    eta_target_m=2.0
)

cols = [
    "site_name", "lat", "lon",
    "L_m", "h_m",
    "U_required_mps", "U_required_kt",
    "Cd_at_solution"
]
cols = [c for c in cols if c in df_out.columns]

# Write CSV
df_out[cols].to_csv(out_csv, index=False)

# Print sorted table
table = df_out[cols].sort_values("U_required_mps").reset_index(drop=True)
with pd.option_context("display.width", 140):
    print(table.to_string(index=False, float_format=lambda x: f"{x:0.2f}"))

print(f"\nWrote: {out_csv}")

         site_name    lat    lon    L_m  h_m  U_required_mps  U_required_kt  Cd_at_solution
           Pamlico  35.23 -76.05 100000 2.50           16.28          31.65            0.00
    Chilika Lagoon  19.70  85.26  51000 1.50           17.30          33.62            0.00
       N. Padre Is  27.47 -97.33  60000 2.00           18.12          35.21            0.00
    Vistula Lagoon  54.52  19.82  75000 2.70           18.64          36.23            0.00
          Padre Is  26.71 -97.43  81000 3.00           18.83          36.61            0.00
        S Padre Is  26.18 -97.23  81000 3.00           18.83          36.61            0.00
     Terrebone Bay  29.19 -90.52  40000 1.50           18.92          36.78            0.00
     Galveston Bay  29.48 -94.83  76000 3.00           19.28          37.48            0.00
         Matagorda  28.24 -96.73  75000 3.00           19.37          37.66            0.00
         Albemarle  36.06 -76.09  75000 3.00           19.37          37.66     